In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from torch import nn,optim
import torch

import algo
import json

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
import numpy as np
import pandas as pd

import attack
import random




2023-02-26 22:16:36.569251: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

class Train_args():

    def __init__(self, learning_rate, weight_decay, epoch):
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.epoch = epoch
         
def train_attack_model(model, train_data, train_target, train_args):

    optimizer = optim.Adam(model.parameters(), lr=train_args.learning_rate, weight_decay=train_args.weight_decay)
    
    model.train()
    optimizer.zero_grad()
    output = model(train_data)
    loss = nn.CrossEntropyLoss()(output, train_target.to(torch.long))
    loss.backward(retain_graph=True)
    optimizer.step()

    return model

def attack_evaluation(model, x, y, dev="cpu", extended=False):

    model.eval()

    with torch.no_grad():
        output =  model(x)
        out_target = output.argmax(1, keepdim=True)
        correct = out_target.to(dev).eq(y.to(dev).view_as(out_target.to(dev))).sum().item()
        acc = correct/y.shape[0]

        predicted_positive = output.argmax(1, keepdim=True) == 1
        labeled_positive = y == 1
        tp = predicted_positive.to(dev) * labeled_positive.to(dev).view_as(out_target)
        tp_count = tp.to(dev).sum().item()
        
        if predicted_positive.to(dev).sum().item() != 0:
            pre = tp_count / predicted_positive.to(dev).sum().item()
        else:
            pre = 0
        if labeled_positive.to(dev).sum().item() !=0:
            rec = tp_count / labeled_positive.to(dev).sum().item()
        else:
            rec = 0
    if extended:
        predicted_negative = output.argmax(1, keepdim=True) == 0
        labeled_negative = y == 0
        tn = predicted_negative.to(dev) * labeled_negative.to(dev).view_as(out_target)
        tn_count = tn.to(dev).sum().item()

        fp_count = predicted_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        fn_count = labeled_positive.to(dev).sum().item() - tp.to(dev).sum().item()
        
        return acc, pre, rec, tp_count, tn_count, fp_count, fn_count
    else:
        return acc, pre, rec

class model_params(object):

    def __init__(self, model_name):
        self.model_name      = model_name

In [3]:
# raw_data = pd.read_csv('../datasets/dataset_purchase', )
# y=raw_data['63']
# X=raw_data.drop('63', axis=1)
# y =  y.replace(100, 0)
# print(y.nunique())

# X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.1, random_state=42)
# print(X_train.shape, x_shadow.shape)

In [4]:
data = pd.read_csv('../datasets.nosync/texas/100/feats')
target = pd.read_csv('../datasets.nosync/texas/100/labels')
data['label'] = target
data.shape

(67329, 6170)

In [5]:
target = target.replace(100, 0)
data['label'] = target

In [261]:
#Texas
y = data['label']
X = data.drop('label', axis=1)
# scaler = StandardScaler()
# X = scaler.fit_transform(X)
tr_size = 10000
X_train, X_test, y_train, y_test = train_test_split(X[0:30000], y[0:30000], train_size=0.5, random_state=0)
x_target_train = np.array(X_train)[:tr_size]
y_target_train = np.array(y_train)[:tr_size]
x_target_test = np.array(X_test)[:tr_size]
y_target_test = np.array(y_test)[:tr_size]

x_shadow = X[30000:]
y_shadow = y[30000:]

In [264]:
np.save('data/texas_x_target_train', x_target_train)
np.save('data/texas_y_target_train', y_target_train)
np.save('data/texas_x_target_test', x_target_test)
np.save('data/texas_y_target_test', y_target_test)



In [266]:
def load_texas():

    x_target_train = np.load(f'data/texas_x_target_train.npy')
    y_target_train = np.load(f'data/texas_y_target_train.npy')
    x_target_test = np.load(f'data/texas_x_target_test.npy')
    y_target_test = np.load(f'data/texas_y_target_test.npy')

    return x_target_train, y_target_train, x_target_test, y_target_test

x_target_train, y_target_train, x_target_test, y_target_test = load_texas()

In [257]:
model = algo.LogisticRegression_DPSGD()

model.n_classes      = len(np.unique(y))
model.alpha          = 0.001
model.max_iter       = 100
model.lambda_        = 1e-5
model.tolerance      = 1e-5
model.DP             = False
model.L              = 1
model.epsilon        = 1

In [258]:
rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
random.seed(rand_seed)

X,y = model.init_theta(x_target_train, y_target_train)
model.train(X,y)
model.evaluate(x_target_train, y_target_train, acc=True)
model.evaluate(x_target_test, y_target_test, acc=True)
            

The accuracy of the model : 54.2 %
The accuracy of the model : 27.500000000000004 %


0.275

In [259]:
target_model.evaluate(x_target_train, y_target_train, acc=True)
target_model.evaluate(x_target_test, y_target_test, acc=True)

The accuracy of the model : 54.2 %
The accuracy of the model : 27.500000000000004 %


0.275

In [144]:
# model10k =model1k

In [94]:
# model1k = model

In [96]:
s_ms = {}
number_of_sms = 5
shadow_size = 10000
shadow_batch_size = int(shadow_size/number_of_sms)

x_shadow_train = np.array(x_shadow[:shadow_size])
y_shadow_train = np.array(y_shadow[:shadow_size])
x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

for i in range(number_of_sms):
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    shadow_model = algo.LogisticRegression_DPSGD()

    shadow_model.n_classes      = len(np.unique(y_target_test))
    shadow_model.alpha          = 0.001
    shadow_model.max_iter       = 100
    shadow_model.lambda_        = 1e-5
    shadow_model.tolerance      = 1e-5
    shadow_model.DP             = False

    X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
    shadow_model.SGD(X,y)
    print('Shadow model: ', i)
    shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end], acc=True)
    shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end], acc=True)
    s_ms[i] = shadow_model

/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_52169/513802048.py:7: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_train = np.array(y_shadow[:shadow_size])
/var/folders/66/0p0lnj8d6zj532q0yvm1zt580000gn/T/ipykernel_52169/513802048.py:9: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])


Shadow model:  0
The accuracy of the model : 65.2 %
The accuracy of the model : 36.7 %
Shadow model:  1
The accuracy of the model : 63.6 %
The accuracy of the model : 37.5 %
Shadow model:  2
The accuracy of the model : 61.0 %
The accuracy of the model : 32.300000000000004 %
Shadow model:  3
The accuracy of the model : 60.699999999999996 %
The accuracy of the model : 33.0 %
Shadow model:  4
The accuracy of the model : 63.0 %
The accuracy of the model : 37.4 %


In [97]:
shadow_train_pred = []
shadow_test_pred = []

for i in range(number_of_sms): 
    batch_start = i*shadow_batch_size
    batch_end = (i+1)*shadow_batch_size
    
    train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
    test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
    shadow_train_pred.append(train_prediciton)
    shadow_test_pred.append(test_prediciton)
    
    

In [98]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(np.arange(model.n_classes).reshape(-1, 1))

y_shadow_train_ohe = ohe.transform(y_shadow_train.reshape(-1,1)) #encoode the target values
y_shadow_test_ohe = ohe.transform(y_shadow_test.reshape(-1,1)) #encoode the target values


In [99]:
y_shadow_train_ohe.shape

(10000, 100)

In [100]:
y_shadow_test_ohe.shape

(10000, 100)

In [101]:
sh_train_pred = np.concatenate(shadow_train_pred)
sh_test_pred = np.concatenate(shadow_test_pred)

# members
labels = np.ones(sh_train_pred.shape[0])
# non-members
test_labels = np.zeros(sh_test_pred.shape[0])

x_1 = np.concatenate((sh_train_pred, sh_test_pred))
x_2 = np.concatenate((y_shadow_train_ohe, y_shadow_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))


In [102]:
attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)

In [103]:
# k=100
attack_params = model_params('attack_model')
attack_params.learning_rate = 0.001
attack_params.weight_decay = 1e-5
attack_params.epoch = 400
attack_params.h_neurons=64
attack_params.do=0

for attack_params.learning_rate in [0.001, 0.01]:
    for attack_params.h_neurons in [32,64,128]:
        for attack_params.do in [0, 0.5]:
            # attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
            # attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
            attack_train_args = Train_args(attack_params.learning_rate, attack_params.weight_decay, attack_params.epoch)
            attack_model = Net_attack(attack_params.h_neurons, attack_params.do, input_size=attack_train_data.shape[1])
            for epoch in range(attack_train_args.epoch):
                attack_model = attack.train_attack_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
                print(f'epoch {epoch}: {attack_evaluation(attack_model, attack_train_data, attack_train_target)}' )        
                
                train_res = attack_evaluation(attack_model, attack_train_data, attack_train_target)
                test_res = attack.mi_attack_test(model, attack_model, x_target_train, y_target_train, x_target_test, y_target_test)

                attack_params.number_of_sms = number_of_sms
                attack_params.shadow_size = shadow_size

                attack_params.train_acc, attack_params.train_pre, attack_params.train_rec = train_res
                attack_params.test_acc, attack_params.test_pre, attack_params.test_rec = test_res

                a_param = dict(shadow_model.__dict__)
                a_param.pop('theta')
                a_param.pop('pred_func')
                a_param.pop('accuracy')
                a_param.update(dict(attack_params.__dict__))
                a_param

                k+=1
                at_path = 'mia/texas/attack_model'+str(k)
                torch.save(attack_model, at_path)
                with open(at_path+'_params.json', 'w') as file:
                    json.dump(a_param, file)


epoch 0: (0.5, 0, 0.0)
epoch 1: (0.5, 0, 0.0)
epoch 2: (0.5, 0, 0.0)
epoch 3: (0.5, 0, 0.0)
epoch 4: (0.5, 0, 0.0)
epoch 5: (0.5, 0, 0.0)
epoch 6: (0.5, 0, 0.0)
epoch 7: (0.5, 0, 0.0)
epoch 8: (0.50005, 1.0, 0.0001)
epoch 9: (0.50045, 1.0, 0.0009)
epoch 10: (0.5018, 0.95, 0.0038)
epoch 11: (0.50325, 0.891566265060241, 0.0074)
epoch 12: (0.5066, 0.7894736842105263, 0.018)
epoch 13: (0.5083, 0.7207446808510638, 0.0271)
epoch 14: (0.51125, 0.6817447495961227, 0.0422)
epoch 15: (0.51825, 0.6574633304572908, 0.0762)
epoch 16: (0.52665, 0.6427423674343867, 0.12)
epoch 17: (0.53395, 0.6303262955854126, 0.1642)
epoch 18: (0.54405, 0.6213164417515836, 0.2256)
epoch 19: (0.55015, 0.6038947586492646, 0.2915)
epoch 20: (0.5514, 0.5805390159824506, 0.3705)
epoch 21: (0.56155, 0.5828956228956229, 0.4328)
epoch 22: (0.5655, 0.5758277379022922, 0.4974)
epoch 23: (0.5616, 0.5650063317855635, 0.5354)
epoch 24: (0.5583, 0.5585341365461848, 0.5563)
epoch 25: (0.55765, 0.5565806261654725, 0.5671)
epoch 26:

epoch 178: (0.62115, 0.6273520445705876, 0.5968)
epoch 179: (0.6216, 0.6243353783231084, 0.6106)
epoch 180: (0.622, 0.6285292878213232, 0.5966)
epoch 181: (0.62185, 0.6247057619486235, 0.6104)
epoch 182: (0.6228, 0.6296452702702703, 0.5964)
epoch 183: (0.6226, 0.6256147540983606, 0.6106)
epoch 184: (0.6236, 0.6305725755334882, 0.5969)
epoch 185: (0.62315, 0.6262688403568133, 0.6108)
epoch 186: (0.6242, 0.6310679611650486, 0.598)
epoch 187: (0.62385, 0.6270647378680619, 0.6112)
epoch 188: (0.62475, 0.6316900665048031, 0.5984)
epoch 189: (0.6246, 0.6282156822391438, 0.6105)
epoch 190: (0.62565, 0.6327522451135763, 0.5989)
epoch 191: (0.62515, 0.6284248332478194, 0.6124)
epoch 192: (0.6259, 0.6330585499894315, 0.599)
epoch 193: (0.62585, 0.629728893928461, 0.6109)
epoch 194: (0.6256, 0.6330790421699513, 0.5975)
epoch 195: (0.62635, 0.6301369863013698, 0.6118)
epoch 196: (0.626, 0.6337295690936107, 0.5971)
epoch 197: (0.62735, 0.6310859495625322, 0.6131)
epoch 198: (0.62625, 0.633725240970

epoch 348: (0.65675, 0.6715176715176715, 0.6137)
epoch 349: (0.657, 0.6661727349703641, 0.6294)
epoch 350: (0.657, 0.6718852638493541, 0.6137)
epoch 351: (0.6573, 0.6666313559322034, 0.6293)
epoch 352: (0.6572, 0.6722550953320184, 0.6135)
epoch 353: (0.6574, 0.6666313783612111, 0.6297)
epoch 354: (0.65735, 0.6723627998685507, 0.6138)
epoch 355: (0.6579, 0.6674798472634705, 0.6293)
epoch 356: (0.6578, 0.6728368017524644, 0.6143)
epoch 357: (0.65815, 0.6677272245200976, 0.6296)
epoch 358: (0.6584, 0.6736461302345976, 0.6145)
epoch 359: (0.6587, 0.668399830220713, 0.6299)
epoch 360: (0.65825, 0.6735007126411577, 0.6143)
epoch 361: (0.6592, 0.6688229056203605, 0.6307)
epoch 362: (0.6584, 0.6736461302345976, 0.6145)
epoch 363: (0.65945, 0.6692854867820363, 0.6304)
epoch 364: (0.65865, 0.6742832033395584, 0.6138)
epoch 365: (0.66005, 0.6699224970803694, 0.631)
epoch 366: (0.6591, 0.6746049165935031, 0.6147)
epoch 367: (0.6603, 0.6704231341696789, 0.6306)
epoch 368: (0.6597, 0.675455943748626

epoch 124: (0.60095, 0.6039328734685473, 0.5866)
epoch 125: (0.6009, 0.60380658436214, 0.5869)
epoch 126: (0.6004, 0.6015166835187058, 0.5949)
epoch 127: (0.60185, 0.604708543230184, 0.5882)
epoch 128: (0.60215, 0.6039906342257966, 0.5933)
epoch 129: (0.602, 0.6066053511705686, 0.5804)
epoch 130: (0.6033, 0.6052149113872479, 0.5942)
epoch 131: (0.603, 0.6088104796112402, 0.5763)
epoch 132: (0.60475, 0.6068114612011828, 0.5951)
epoch 133: (0.60415, 0.6064819548103466, 0.5932)
epoch 134: (0.60365, 0.6040140491721023, 0.6019)
epoch 135: (0.603, 0.6051664284255667, 0.5927)
epoch 136: (0.60205, 0.6008299575140796, 0.6081)
epoch 137: (0.6032, 0.6047503045066992, 0.5958)
epoch 138: (0.60425, 0.6078411089272784, 0.5876)
epoch 139: (0.60535, 0.6058687569088533, 0.6029)
epoch 140: (0.60505, 0.6055036657627799, 0.6029)
epoch 141: (0.6063, 0.6079626244160065, 0.5986)
epoch 142: (0.6068, 0.6092918542775276, 0.5954)
epoch 143: (0.60695, 0.6091215182124273, 0.597)
epoch 144: (0.6078, 0.60764929099261

epoch 295: (0.64655, 0.6588617886178861, 0.6078)
epoch 296: (0.6478, 0.6538301415487094, 0.6282)
epoch 297: (0.6483, 0.6607065452969224, 0.6097)
epoch 298: (0.6492, 0.6582184517497349, 0.6207)
epoch 299: (0.6483, 0.6553204859656473, 0.6257)
epoch 300: (0.6472, 0.6556683587140439, 0.62)
epoch 301: (0.6465, 0.6586871750433275, 0.6081)
epoch 302: (0.6456, 0.6586056644880174, 0.6046)
epoch 303: (0.6474, 0.6551905664350389, 0.6223)
epoch 304: (0.64725, 0.6566322731624296, 0.6173)
epoch 305: (0.64735, 0.6631963672610477, 0.5988)
epoch 306: (0.6476, 0.6563228129633553, 0.6197)
epoch 307: (0.64895, 0.6616035586416404, 0.6098)
epoch 308: (0.64775, 0.658343157217876, 0.6143)
epoch 309: (0.6487, 0.6618415324336091, 0.6081)
epoch 310: (0.648, 0.6644809957768393, 0.5979)
epoch 311: (0.6484, 0.6580068143100511, 0.618)
epoch 312: (0.6491, 0.6639182058047494, 0.6039)
epoch 313: (0.6481, 0.6564546799070357, 0.6214)
epoch 314: (0.6488, 0.6626940739120927, 0.6061)
epoch 315: (0.6493, 0.6645360370288738, 

...

epoch 363: (0.6258, 0.7368222891566265, 0.3914)
epoch 364: (0.6095, 0.5734702093397746, 0.8547)
epoch 365: (0.62445, 0.7404830917874397, 0.3832)
epoch 366: (0.60895, 0.5734213895815082, 0.8509)
epoch 367: (0.6256, 0.7370705926764817, 0.3905)
epoch 368: (0.6095, 0.5741769407939303, 0.8476)
epoch 369: (0.6261, 0.7389162561576355, 0.39)
epoch 370: (0.61065, 0.5752056004893632, 0.8463)
epoch 371: (0.6267, 0.7400530503978779, 0.3906)
epoch 372: (0.6103, 0.5747188727814659, 0.8484)
epoch 373: (0.6266, 0.7369011976047904, 0.3938)
epoch 374: (0.60955, 0.5740252719778364, 0.8495)
epoch 375: (0.6273, 0.7480514419329696, 0.3839)
epoch 376: (0.608, 0.5728057165970069, 0.8497)
epoch 377: (0.6273, 0.7475690392843252, 0.3844)
epoch 378: (0.60835, 0.5726547307718098, 0.854)
epoch 379: (0.62675, 0.7433288539066999, 0.3872)
epoch 380: (0.60755, 0.5718484868728706, 0.856)
epoch 381: (0.6254, 0.743495145631068, 0.3829)
epoch 382: (0.6081, 0.5725016767270288, 0.8536)
epoch 383: (0.6258, 0.7437039907012786,

epoch 136: (0.59955, 0.6805731906403047, 0.3752)
epoch 137: (0.57245, 0.5463798732475513, 0.8535)
epoch 138: (0.59945, 0.6807196074868254, 0.3746)
epoch 139: (0.57245, 0.5464095829863558, 0.853)
epoch 140: (0.59965, 0.6810830456114847, 0.3748)
epoch 141: (0.5725, 0.5464267418032787, 0.8533)
epoch 142: (0.59945, 0.6810486073184052, 0.3741)
epoch 143: (0.57225, 0.546263687007748, 0.8531)
epoch 144: (0.5997, 0.6816029143897996, 0.3742)
epoch 145: (0.5725, 0.546462445526788, 0.8527)
epoch 146: (0.5996, 0.6816192560175055, 0.3738)
epoch 147: (0.5726, 0.5465444287729196, 0.8525)
epoch 148: (0.59925, 0.6809480401093893, 0.3735)
epoch 149: (0.57265, 0.5465854440525809, 0.8524)
epoch 150: (0.59945, 0.6812465828321487, 0.3738)
epoch 151: (0.5727, 0.5466204950622034, 0.8524)
epoch 152: (0.5993, 0.6814692982456141, 0.3729)
epoch 153: (0.57265, 0.5466093539488035, 0.852)
epoch 154: (0.5992, 0.6810218978102189, 0.3732)
epoch 155: (0.5724, 0.5464459840903259, 0.8518)
epoch 156: (0.59925, 0.6811461945

epoch 306: (0.5987, 0.6826424870466321, 0.3689)
epoch 307: (0.5739, 0.5474082627662304, 0.8533)
epoch 308: (0.5983, 0.682239525398591, 0.368)
epoch 309: (0.5738, 0.5473501860644168, 0.8531)
epoch 310: (0.59845, 0.6826191801150064, 0.368)
epoch 311: (0.574, 0.547490694390964, 0.8531)
epoch 312: (0.5986, 0.6827279466271312, 0.3684)
epoch 313: (0.57385, 0.5473913880510813, 0.853)
epoch 314: (0.59845, 0.6832991994042078, 0.367)
epoch 315: (0.57405, 0.5475380368492008, 0.8529)
epoch 316: (0.59865, 0.6836715695401229, 0.3672)
epoch 317: (0.57385, 0.5474035560690673, 0.8528)
epoch 318: (0.59865, 0.6836032011911409, 0.3673)
epoch 319: (0.5739, 0.5474386955963538, 0.8528)
epoch 320: (0.5986, 0.6835443037974683, 0.3672)
epoch 321: (0.5738, 0.5473623411628803, 0.8529)
epoch 322: (0.59865, 0.6838084591019191, 0.367)
epoch 323: (0.5739, 0.5474326059050064, 0.8529)
epoch 324: (0.5989, 0.6839657738095238, 0.3677)
epoch 325: (0.57395, 0.5474738396353598, 0.8528)
epoch 326: (0.59865, 0.6838769804287046

epoch 78: (0.6074, 0.7329718004338395, 0.3379)
epoch 79: (0.584, 0.5517815312538528, 0.8951)
epoch 80: (0.60645, 0.723775488753416, 0.3443)
epoch 81: (0.5856, 0.5530885636318531, 0.8918)
epoch 82: (0.6112, 0.7222222222222222, 0.3614)
epoch 83: (0.58175, 0.5502798450089181, 0.8947)
epoch 84: (0.61105, 0.7248430856448674, 0.358)
epoch 85: (0.5805, 0.5489718943910451, 0.9024)
epoch 86: (0.6103, 0.7215749296906389, 0.3592)
epoch 87: (0.57975, 0.5483773127085229, 0.904)
epoch 88: (0.61225, 0.7344892416962607, 0.3516)
epoch 89: (0.57865, 0.547537020247809, 0.9059)
epoch 90: (0.6115, 0.736228813559322, 0.3475)
epoch 91: (0.5804, 0.548804176277771, 0.9041)
epoch 92: (0.6123, 0.7350355797404772, 0.3512)
epoch 93: (0.58265, 0.5506713260989516, 0.8982)
epoch 94: (0.61225, 0.7294093603106478, 0.3569)
epoch 95: (0.58195, 0.5504525026165117, 0.8941)
epoch 96: (0.6144, 0.7292585170340682, 0.3639)
epoch 97: (0.5791, 0.5483673718967836, 0.8968)
epoch 98: (0.61595, 0.7185673892554194, 0.3812)
epoch 99: 

epoch 249: (0.59095, 0.556592620247651, 0.8945)
epoch 250: (0.61865, 0.7467248908296943, 0.3591)
epoch 251: (0.59085, 0.5567706055114666, 0.891)
epoch 252: (0.61885, 0.7457101509199917, 0.3607)
epoch 253: (0.59115, 0.5565061062550369, 0.8977)
epoch 254: (0.6193, 0.7340133385641427, 0.3742)
epoch 255: (0.5935, 0.5585913021681915, 0.8914)
epoch 256: (0.6194, 0.7366230677764566, 0.3717)
epoch 257: (0.59235, 0.5580051504302493, 0.8884)
epoch 258: (0.6207, 0.743052758759565, 0.369)
epoch 259: (0.59275, 0.5583663709017683, 0.8873)
epoch 260: (0.61815, 0.743056984159638, 0.3612)
epoch 261: (0.59375, 0.5594822663536577, 0.8818)
epoch 262: (0.6189, 0.7429505516959543, 0.3636)
epoch 263: (0.59165, 0.5576668973762033, 0.8863)
epoch 264: (0.6194, 0.7424857839155159, 0.3656)
epoch 265: (0.5916, 0.557523235367998, 0.8878)
epoch 266: (0.62085, 0.7326275264677574, 0.3806)
epoch 267: (0.59245, 0.5581775847964257, 0.887)
epoch 268: (0.62195, 0.7336654531519449, 0.3829)
epoch 269: (0.59155, 0.55740171797

# Attack models texas

In [248]:
import os
path = 'mia/texas/best_mia/'
aparams = {}
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if ".json" in file:
            with open(r+'/'+file) as json_file:
                aparams[r+file.replace('.json', '')] = json.load(json_file)
            
for r,d,f in os.walk(path):
    for file in f:
        if ".json" not in file and '.DS_Store' not in file:
            ams[r+file] = Net_attack(h_neurons=aparams[r+file]['h_neurons'], do=aparams[r+file]['do'], input_size=14)
            ams[r+file] = torch.load(r+'/'+file)      
        

In [249]:
attack_dict = attack.test_mi_attack(ams, model, x_target_train, y_target_train, x_target_test, y_target_test)
attack_dict 

{'attack_acc_mean': 0.5413636363636365,
 'attack_acc_std': 0.007413412013772774,
 'attack_pre_mean': 0.9583333333333333,
 'attack_pre_std': 0.05892556509887896,
 'attack_rec_mean': 0.08727272727272727,
 'attack_rec_std': 0.016564424689353267}

In [260]:
attack_dict = attack.test_mi_attack(ams, model, x_target_train[:1000], y_target_train[:1000], x_target_test[:1000], y_target_test[:1000])
attack_dict 

{'attack_acc_mean': 0.6455909090909091,
 'attack_acc_std': 0.0034299362690751855,
 'attack_pre_mean': 0.700552255210943,
 'attack_pre_std': 0.007573150520431135,
 'attack_rec_mean': 0.5088181818181818,
 'attack_rec_std': 0.008277221556429125}

In [198]:
# attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
a_10k = {}
a_1k = {}
for m in ams:
    model = ams[m]
    test_acc, test_pre, test_rec = attack.mi_attack_test(model10k, model, x_target_train, y_target_train, x_target_test, y_target_test)
    a_10k[m] = test_acc, test_pre, test_rec
    test_acc, test_pre, test_rec = attack.mi_attack_test(target_model, model, x_target_train[:1000], y_target_train[:1000], x_target_test[:1000], y_target_test[:1000])
    a_1k[m] = test_acc, test_pre, test_rec

In [227]:
for i,m in enumerate(best_models):
    print(m)
    torch.save(ams[m], 'mia/texas/best_mia/attack_model_'+str(i))
    with open('mia/texas/best_mia/attack_model_'+str(i)+'.json', 'w') as file:
                    json.dump(aparams[m], file)

mia/texas/attack_model9699
mia/texas/attack_model9697
mia/texas/attack_model9665
mia/texas/attack_model9691
mia/texas/attack_model9683
mia/texas/attack_model9669
mia/texas/attack_model9693
mia/texas/attack_model9695
mia/texas/attack_model9657
mia/texas/attack_model9681
mia/texas/attack_model9689


In [206]:
m

'mia/texas/attack_model9699'

In [201]:
best_models

[]

In [235]:
best_res1k = {}
best_res10k = {}
best_res10k['acc'] = []
best_res10k['pre'] = []
best_res10k['rec'] = []
best_res10k['f1'] = []

best_res1k['acc'] = []
best_res1k['pre'] = []
best_res1k['rec'] = []
best_res1k['f1'] = []
best_models = []
for i in a_1k:
    if a_10k[i][0]>0.55 and a_1k[i][0]>0.64:
        best_models.append(i)
        f1_1 = 2*a_1k[i][1]*a_1k[i][2]/(a_1k[i][1]+a_1k[i][2])
        print('1K: ',a_1k[i], f1)
        f1_10 = 2*a_10k[i][1]*a_10k[i][2]/(a_10k[i][1]+a_10k[i][2])
        print('10K:', a_10k[i], f1)
        best_res10k['acc'].append(a_10k[i][0])
        best_res10k['pre'].append(a_10k[i][1])
        best_res10k['rec'].append(a_10k[i][2])
        best_res10k['f1'].append(f1_10)
        
        best_res1k['acc'].append(a_1k[i][0])
        best_res1k['pre'].append(a_1k[i][1])
        best_res1k['rec'].append(a_1k[i][2])
        best_res1k['f1'].append(f1_1)

1K:  (0.6405, 0.6937931034482758, 0.503) 0.6085054252712635
10K: (0.5549, 0.5423807318202871, 0.7026) 0.6085054252712635
1K:  (0.6495, 0.7151079136690648, 0.497) 0.6085054252712635
10K: (0.5539, 0.5420240137221269, 0.6952) 0.6085054252712635
1K:  (0.6425, 0.6949384404924761, 0.508) 0.6085054252712635
10K: (0.5537, 0.5421440904096688, 0.6908) 0.6085054252712635
1K:  (0.649, 0.6991978609625669, 0.523) 0.6085054252712635
10K: (0.5537, 0.54156346749226, 0.6997) 0.6085054252712635
1K:  (0.6455, 0.6958277254374159, 0.517) 0.6085054252712635
10K: (0.55295, 0.5413123195755637, 0.6938) 0.6085054252712635
1K:  (0.642, 0.7005649717514124, 0.496) 0.6085054252712635
10K: (0.5529, 0.5416469847268147, 0.688) 0.6085054252712635
1K:  (0.648, 0.7, 0.518) 0.6085054252712635
10K: (0.55365, 0.5415472779369628, 0.6993) 0.6085054252712635
1K:  (0.651, 0.7138810198300283, 0.504) 0.6085054252712635
10K: (0.5533, 0.5416341196688017, 0.6934) 0.6085054252712635
1K:  (0.642, 0.6908602150537635, 0.514) 0.6085054252

In [236]:
print(np.mean(best_res1k['acc']), np.mean(best_res1k['pre']), np.mean(best_res1k['rec']), np.mean(best_res1k['f1']))
print(np.mean(best_res10k['acc']), np.mean(best_res10k['pre']), np.mean(best_res10k['rec']), np.mean(best_res10k['f1']))

0.6455909090909091 0.700552255210943 0.508818181818182 0.5894076353621919
0.5533818181818181 0.5416201733775552 0.6946727272727272 0.608666034689381


In [237]:
attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
print(attack_dict)
attack_dict = attack.test_mi_attack(ams, model10k, x_target_train, y_target_train, x_target_test, y_target_test)
print(attack_dict)

{'attack_acc_mean': 0.5129863636363635, 'attack_acc_std': 0.0010101959551337707, 'attack_pre_mean': 0.5281946482817765, 'attack_pre_std': 0.00191616621491754, 'attack_rec_mean': 0.24327272727272728, 'attack_rec_std': 0.008797839704784038}
{'attack_acc_mean': 0.5533818181818182, 'attack_acc_std': 0.0006719282724411607, 'attack_pre_mean': 0.5416201733775551, 'attack_pre_std': 0.0004041122448942372, 'attack_rec_mean': 0.6946727272727272, 'attack_rec_std': 0.004681897616272309}


In [116]:

for i in aparams:
    if aparams[i]['test_acc']>0.55:
        print(aparams[i])
        print('train:', aparams[i]['train_acc'])
        print('test: ', aparams[i]['test_acc'])

{'n_classes': 100, 'alpha': 0.001, 'max_iter': 100, 'lambda_': 1e-05, 'tolerance': 1e-05, 'sgdDP': False, 'L': 1, 'C': 1, 'epsilon': 1, 'delta': 1e-05, 'sigma': 0, 'DP': False, 'model_name': 'attack_model', 'learning_rate': 0.001, 'weight_decay': 1e-05, 'epoch': 400, 'h_neurons': 128, 'do': 0.5, 'number_of_sms': 5, 'shadow_size': 10000, 'train_acc': 0.6932, 'train_pre': 0.724808005585292, 'train_rec': 0.6229, 'test_acc': 0.5522, 'test_pre': 0.543471019320453, 'test_rec': 0.6526}
train: 0.6932
test:  0.5522
{'n_classes': 100, 'alpha': 0.001, 'max_iter': 100, 'lambda_': 1e-05, 'tolerance': 1e-05, 'sgdDP': False, 'L': 1, 'C': 1, 'epsilon': 1, 'delta': 1e-05, 'sigma': 0, 'DP': False, 'model_name': 'attack_model', 'learning_rate': 0.001, 'weight_decay': 1e-05, 'epoch': 400, 'h_neurons': 128, 'do': 0, 'number_of_sms': 5, 'shadow_size': 10000, 'train_acc': 0.69055, 'train_pre': 0.71985692857967, 'train_rec': 0.6239, 'test_acc': 0.55115, 'test_pre': 0.5413667610190053, 'test_rec': 0.6694}
trai

In [73]:
train_res = attack_evaluation(attack_model, attack_train_data, attack_train_target)
test_res = attack.mi_attack_test(model, attack_model, x_target_train, y_target_train, x_target_test, y_target_test)

attack_params.number_of_sms = number_of_sms
attack_params.shadow_size = shadow_size

attack_params.train_acc, attack_params.train_pre, attack_params.train_rec = train_res
attack_params.test_acc, attack_params.test_pre, attack_params.test_rec = test_res

a_param = dict(shadow_model.__dict__)
a_param.pop('theta')
a_param.pop('pred_func')
a_param.pop('accuracy')
a_param.update(dict(attack_params.__dict__))
a_param

k+=1
at_path = 'mia/loan/attack_model'+str(k)
torch.save(attack_model, at_path)
with open(at_path+'_params.json', 'w') as file:
    json.dump(a_param, file)
        

AttributeError: 'LogisticRegression_DPSGD' object has no attribute 'theta'

In [ ]:
sh_path = f'mia/shms{number_of_sms}_shtrsize{shadow_size}_shlr{shadow_model.alpha}_shiter{int(shadow_model.max_iter/shadow_batch_size)}_shreg{shadow_model.lambda_}/'      
os.mkdir(sh_path)

torch.save(attack_train_data, sh_path+'attack_train_data.pt')
torch.save(attack_train_target, sh_path+'attack_train_target.pt')

at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
torch.save(attack_model, at_path)


In [81]:
train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
train_acc, train_pre, train_rec 

(0.6107, 0.5711531045121481, 0.8886)

In [82]:
test_acc, test_pre, test_rec = attack.mi_attack_test(model, attack_model, x_target_train, y_target_train, x_target_test, y_target_test)
test_acc, test_pre, test_rec

NameError: name 'attack_test_data' is not defined

In [ ]:
# MI optimization

In [ ]:
ams

In [ ]:
rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

epo = [200]
wd = [1e-5, 1e-6]
ams = {}
for ep in epo:
    for w_d in wd:
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=ep)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        for epoch in range(attack_train_args.epoch):

            attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
            ams[(l_r,w_d)] = attack_model
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        torch.save(attack_model, at_path)


In [ ]:
epo = [200,500]
wd = [1e-5, 1e-6]

for ep in epo:
    for w_d in wd:
        print(ep, w_d)
        attack_train_args = Train_args(learning_rate=l_r, weight_decay=w_d, epoch=500)
        attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
        at_path = sh_path+f'attack_model_aneur{attack_model.h_neurons}_ado{attack_model.do}_alr{attack_train_args.learning_rate}_alreg{attack_train_args.weight_decay}_aepoch{attack_train_args.epoch}'
        attack_model = torch.load(at_path)
        
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print('Train: ', train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print('Test: ' , test_acc, test_pre, test_rec)


In [ ]:
for l_r in lr:
    for w_d in wd:
        ams[(l_r,w_d)]
        train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)
        print("Train acc pre rec: ", train_acc, train_pre, train_rec)
        test_acc, test_pre, test_rec = attack_evaluation(attack_model, attack_test_data, attack_test_target)
        print("Test acc pre rec: ", test_acc, test_pre, test_rec)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
shadow_size = 20000
shadow_clf = LogisticRegression(random_state=1).fit(x_shadow[:shadow_size], y_shadow[:shadow_size])

In [ ]:
print(shadow_clf.score(x_shadow[:shadow_size], y_shadow[:shadow_size]))
print(shadow_clf.score(x_shadow[shadow_size:shadow_size*2], y_shadow[shadow_size:shadow_size*2]))

In [ ]:
shadow_train_pred = shadow_clf.predict_proba(x_shadow[:shadow_size])
shadow_test_pred = shadow_clf.predict_proba(x_shadow[shadow_size:shadow_size*2])
y_shadow_train = y_shadow[:shadow_size]
y_shadow_test = y_shadow[shadow_size:shadow_size*2]

In [ ]:
# members
labels = np.ones(shadow_train_pred.shape[0])
# non-members
test_labels = np.zeros(shadow_test_pred.shape[0])

x_1 = np.concatenate((shadow_train_pred, shadow_test_pred))
x_2 = np.concatenate((y_shadow_train, y_shadow_test)).reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_train_data = np.concatenate((x_1,x_2),axis=1)
attack_train_target = y_new
df = pd.DataFrame(attack_train_data)
df['a_target'] = attack_train_target
df = df.sample(frac = 1)

attack_train_data = np.array(df.drop(['a_target'], axis=1))
attack_train_target = np.array(df['a_target'])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=200)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])


In [ ]:
attack_train_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)   
attack_train_target = torch.tensor(np.array(df['a_target']), dtype=torch.float)
attack_train_args = Train_args(learning_rate=0.001, weight_decay=1e-3, epoch=500)
attack_model = Net_attack(h_neurons=64, do=0, input_size=attack_train_data.shape[1])
for epoch in range(attack_train_args.epoch):
            
    attack_model = train_model(attack_model, attack_train_data, attack_train_target, attack_train_args)
    train_acc, train_pre, train_rec = attack_evaluation(attack_model, attack_train_data, attack_train_target)

In [ ]:
train_acc, train_pre, train_rec 

In [ ]:
train_pred = model.predict(x_target_train, y_target_train)
test_pred = model.predict(x_target_test, y_target_test)

y_target_train_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_train.reshape(-1,1)) #encoode the target values
y_target_test_ohe = OneHotEncoder(sparse=False).fit_transform(y_target_test.reshape(-1,1)) #encoode the target values
    
# members
labels = np.ones(train_pred.shape[0])
# non-members
test_labels = np.zeros(test_pred.shape[0])

x_1 = np.concatenate((train_pred, test_pred))
x_2 = np.concatenate((y_target_train_ohe, y_target_test_ohe))#.reshape((-1, 1))
y_new = np.concatenate((labels, test_labels))

attack_test_data = np.concatenate((x_1,x_2),axis=1)
attack_test_target = y_new
df = pd.DataFrame(attack_test_data)
df['a_target'] = attack_test_target
df = df.sample(frac = 1)

attack_test_data = torch.tensor(np.array(df.drop(['a_target'], axis=1)), dtype=torch.float, requires_grad=True)
attack_test_target = torch.tensor(np.array(df['a_target']))


In [ ]:
test_acc, test_pre, test_rec = attack_evaluation(a_model, attack_test_data, attack_test_target)
test_acc, test_pre, test_rec

In [ ]:
a_model = Net_attack(h_neurons=64, do=0, input_size=100)
a_model = torch.load('attack_model') 

In [ ]:

rand_seed=42
np.random.seed(rand_seed)
# random.seed(rand_seed)
torch.manual_seed(rand_seed)

lr = [0.001]
wd = [1e-4, 1e-6]
tms = {}
for l_r in lr:
    for w_d in wd:
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = 100
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size 
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.epsilon

        X,y = model.init_theta(x_target_train, y_target_train)
        model.SGD(X,y)
        model.evaluate(x_target_train, y_target_train)

In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

import algo
# import attack

# from torch import nn,optim
import torch

rand_seed=42
np.random.seed(rand_seed)
torch.manual_seed(rand_seed)
        

raw_data_path = '../datasets/dataset_purchase'
raw_data = pd.read_csv(raw_data_path)
y=raw_data['63']
X=raw_data.drop('63', axis=1)
y =  y.replace(100, 0)
print('Dataset: ', raw_data_path)
print('Classes in classification task: ', y.nunique())
n_classes = y.nunique()

X_train, x_shadow, y_train, y_shadow = train_test_split(X, y, train_size=0.2, random_state=rand_seed)
print(X_train.shape, x_shadow.shape)

#Target model
X_train_size = 10000
X_test_size = 10000
x_target_train = np.array(X_train[:X_train_size])
y_target_train = np.array(y_train[:X_train_size])
x_target_test = np.array(X_train[X_train_size:X_train_size+X_test_size])
y_target_test = np.array(y_train[X_train_size:X_train_size+X_test_size])
if y_target_test.shape[0]<X_test_size or y_target_train.shape[0]<X_train_size:
    raise ValueError(
            "Not enough traning or test data for the target model")

for L in [1,10,100]:
    for epsilon in np.arange(0,1,0.1):
        model = algo.LogisticRegression_DPSGD()

        model.n_classes      = n_classes
        model.alpha          = 0.001
        model.max_iter       = 100*X_train_size
        model.lambda_        = 1e-3
        model.tolerance      = 10e-5
        model.DP             = True
        model.L              = L
        model.epsilon        = epsilon


        X,y = model.init_theta(x_target_train, y_target_train)
        model.train(X,y)
        model.evaluate(x_target_train, y_target_train, acc=True)
        model.evaluate(x_target_test, y_target_test, acc=True)

        tm_path = f'tm/lr{model.alpha}_iter{int(model.max_iter/X_train_size)}_reg{model.lambda_}_DP{model.DP}'
        if model.DP:
            tm_path += f'_eps{model.epsilon}_L{model.L}'
        np.save(tm_path+'_target_model', model.theta)

        print(tm_path)

    

In [ ]:
#Shadow models
# s_ms = {}
# number_of_sms = 10
# shadow_size = 50000
# shadow_batch_size = int(shadow_size/number_of_sms)

# x_shadow_train = np.array(x_shadow[:shadow_size])
# y_shadow_train = np.array(y_shadow[:shadow_size])
# x_shadow_test = np.array(x_shadow[shadow_size:2*shadow_size])
# y_shadow_test = np.array(y_shadow[shadow_size:2*shadow_size])

# attack.train_shadow_models(number_of_sms,)

# for i in range(number_of_sms):  
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     shadow_model = algo.LogisticRegression_DPSGD()

#     shadow_model.n_classes      = n_classes
#     shadow_model.alpha          = 0.001
#     shadow_model.max_iter       = 100*shadow_batch_size
#     shadow_model.lambda_        = 10e-3
#     shadow_model.tolerance      = 10e-5
#     shadow_model.DP             = False

#     X,y = shadow_model.init_theta(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end] )
#     shadow_model.SGD(X,y)
#     print('Shadow model: ', i)
#     shadow_model.evaluate(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     shadow_model.evaluate(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
#     s_ms[i] = shadow_model

# #Attack model

# shadow_train_pred = []
# shadow_test_pred = []

# for i in range(number_of_sms): 
#     batch_start = i*shadow_batch_size
#     batch_end = (i+1)*shadow_batch_size
    
#     train_prediciton = s_ms[i].predict(x_shadow_train[batch_start:batch_end], y_shadow_train[batch_start:batch_end])
#     test_prediciton = s_ms[i].predict(x_shadow_test[batch_start:batch_end], y_shadow_test[batch_start:batch_end])
    
#     shadow_train_pred.append(train_prediciton)
#     shadow_test_pred.append(test_prediciton)
    

In [178]:
# attack models
from torch import nn

class Net_attack(nn.Module):

    def __init__(self, h_neurons, do, input_size):
        super(Net_attack, self).__init__()
        self.input_size = input_size
        self.h_neurons = h_neurons
        self.do = do
        self.fc1 = nn.Linear(input_size, h_neurons)
        self.fc2 = nn.Linear(h_neurons, 2)
        self.relu = nn.ReLU()
        self.drop = nn.Dropout(do)
        self.softmax = nn.Softmax(dim=1)   

    def forward(self, x):
        x = x.view(-1, self.input_size)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.drop(x)
        x = self.softmax(x)
        return x

path = 'mia'
ams = {}
for r,d,f in os.walk(path):
    for file in f:
        if "best_mi_model" in file:
            ams[file] = Net_attack(h_neurons=64, do=0, input_size=200)
            ams[file] = torch.load(r+'/'+file)      

In [126]:
target_model = model

In [127]:
attack_dict = attack.test_mi_attack(ams, target_model, x_target_train, y_target_train, x_target_test, y_target_test)
    

In [128]:
# 10 K target 
attack_dict

{'attack_acc_mean': 0.5811666666666667,
 'attack_acc_std': 0.005857094463601882,
 'attack_pre_mean': 0.6439540525080872,
 'attack_pre_std': 0.014326959721099111,
 'attack_rec_mean': 0.36600000000000005,
 'attack_rec_std': 0.04052982440952177}

In [129]:
model